In [1]:
import random
import os
import numpy as np


# 시드 정하기!!!!!!!

CFG = {
    'SEED':42,
    'NSPLIT' :10

}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [3]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
import numpy as np

train_df = pd.read_csv(r'C:\A\breastCancer\ml/train.csv')
test_df = pd.read_csv(r"C:\A\breastCancer\ml/test.csv")

# mean, fillna(0)
train_df['size of cancers'] = train_df['size of cancers'].fillna(train_df['size of cancers'].mean())
train_df = train_df.fillna(0)

test_df['size of cancers'] = test_df['size of cancers'].fillna(train_df['size of cancers'].mean())
test_df = test_df.fillna(0)


target_name = ['N_category']

# 명목형 변수
cat_col_names = [
    # 기본
    #'dx','location of cancers', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
    # 0 : 5개 제거(location of cancers,ER,PR,HER2_IHC,BRCA_mutation)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH'
    # 1 : 3개 제거(location of cancers,ER,BRCA_mutation)
    'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC'
    # 2 : 2개 제거((location of cancers,ER)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC','BRCA_mutation'
]


for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    'age','size of cancers','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,num_col_names+cat_col_names]
y = train_df.loc[:,target_name]


In [6]:
"""
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)
"""
X = np.array(X)                 #np.array(df.iloc[:, :-1]) # class 열 제외한 feature 열들 모음 -> array 변환
y = np.array(y)                           #df['class']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [16]:
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

clf = GradientBoostingClassifier(
    max_depth=3,
    max_features=2, 
    n_estimators = 242,
    learning_rate=0.08723275538259107, 
    random_state=CFG['SEED']) # 모델 선언

clf.fit(X_train,y_train) # 모델 학습
y_pred=clf.predict(X_test) # 예측 라벨
accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

accuracy 각 분할의 정확도 기록 : [0.804]
accuracy 평균 정확도 : 0.804
f1_score 각 분할의 정확도 기록 : [0.8136882129277566]
f1_score 평균 정확도 : 0.8136882129277566


In [ ]:
# test prediction

test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)
array = np.array(test_df_id.values)
len(array),type(array)
test_pred = clf.predict(test)
len(test_pred),type(test_pred)
dic_pred={
    'ID' : array,
    'N_category':test_pred
}
df_test_pred = pd.DataFrame(dic_pred)
df_test_pred
f1 = np.mean(f1_score_history)
print(f1)
df_test_pred.to_csv(fr'C:\A\breastCancer\ml\output\ml/tabular_1_{f1}.csv')

In [6]:
# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits =CFG['NSPLIT'] , shuffle = True, random_state = CFG['SEED'])

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

## classififier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = GradientBoostingClassifier(
      max_depth=3, 
      n_estimators = 242
      learning_rate=0.08723275538259107, 
      random_state=CFG['SEED']) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

In [8]:
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = GradientBoostingClassifier(
      max_depth=3, 
      n_estimators = 242,
      learning_rate=0.08723275538259107, 
      random_state=CFG['SEED']) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

accuracy 각 분할의 정확도 기록 : [0.84, 0.74, 0.79, 0.82, 0.79, 0.79, 0.76, 0.82, 0.7, 0.71]
accuracy 평균 정확도 : 0.776
f1_score 각 분할의 정확도 기록 : [0.8431372549019608, 0.74, 0.8000000000000002, 0.82, 0.8173913043478261, 0.7920792079207921, 0.7499999999999999, 0.7857142857142858, 0.7, 0.743362831858407]
f1_score 평균 정확도 : 0.7791684884743272


In [7]:
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = GradientBoostingClassifier(
      max_depth=3, 
      n_estimators = 242
      learning_rate=0.08723275538259107, 
      random_state=CFG['SEED']) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

accuracy 각 분할의 정확도 기록 : [0.82, 0.75, 0.81, 0.82, 0.74, 0.81, 0.77, 0.83, 0.7, 0.73]
accuracy 평균 정확도 : 0.7779999999999999
f1_score 각 분할의 정확도 기록 : [0.8235294117647058, 0.7572815533980582, 0.8155339805825242, 0.816326530612245, 0.7678571428571429, 0.8118811881188118, 0.7676767676767676, 0.7951807228915663, 0.7058823529411764, 0.765217391304348]
f1_score 평균 정확도 : 0.7826367042147346


## test preidiction

In [8]:
test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)

(250, pandas.core.series.Series)

In [9]:
array = np.array(test_df_id.values)
len(array),type(array)

(250, numpy.ndarray)

In [10]:
test_pred = clf.predict(test)
len(test_pred),type(test_pred)

(250, numpy.ndarray)

In [11]:
dic_pred={
    'ID' : array,
    'N_category':test_pred
}

In [12]:
df_test_pred = pd.DataFrame(dic_pred)

In [13]:
df_test_pred

,ID,N_category
0,BC_01_0011,1
1,BC_01_0220,0
2,BC_01_0233,0
3,BC_01_0258,0
4,BC_01_0260,1
...,...,...
245,BC_01_3328,1
246,BC_01_3404,0
247,BC_01_3418,0
248,BC_01_3438,0


In [14]:
f1 = np.mean(f1_score_history)
print(f1)

0.7826367042147346


In [17]:
df_test_pred.to_csv(fr'C:\A\breastCancer\ml\output\ml/tabular_1_{f1}.csv')

In [ ]:
"""import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))


# print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
f1score=f1_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('LightGBM Model f1score score: {0:0.4f}'.format(f1_score(y_pred, y_test)))"""

In [ ]:
"""from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    loss_function='CrossEntropy'
)
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))"""